In [1]:
#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import gpytorch
import botorch 

# BO with 2nd Order Polynomial Prior
### BO Minimizes Emittance*Bmag with 9 Variables (SQ, CQ, SOL, matching quads)

In [2]:
# load injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-16 22:37:15.221343: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# design Twiss parameters 
beamline_info = json.load(open('../configs/beamline_info.json'))
get_twiss0 = beamline_info['Twiss0']

# emit, beta, alpha
twiss0 = {'x': [get_twiss0[0], get_twiss0[2], get_twiss0[4]],
          'y': [get_twiss0[1], get_twiss0[3], get_twiss0[5]]}

beta0_x, alpha0_x = twiss0['x'][1], twiss0['x'][2]
beta0_y, alpha0_y = twiss0['y'][1], twiss0['y'][2]
# print(twiss0['x'])
# print(twiss0['y'])

## Objective Function

In [4]:
# convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

# input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','CQ01:b1_gradient', 'SQ01:b1_gradient',
                 "QA01:b1_gradient", "QA02:b1_gradient", 
                 "QE01:b1_gradient", "QE02:b1_gradient", "QE03:b1_gradient", "QE04:b1_gradient"]
bounds = torch.tensor([[0.46, 0.485], [-0.02, 0.02], [-0.02, 0.02],
                       [-4, -1], [1, 4],
                       [-7,-1], [-1, 7],[-1, 7], [-7, 1]])

# output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(config): 
    """
    D is input space dimensionality
    :param config: input values of opt_var_names, torch.tensor, shape (1, D) 
    """
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, CQ, SQ, matching quads to values from optimization step
    for i in range(config.size(dim=0)):
        x_in[:, Model.loc_in[opt_var_names[i]]] = config[i]

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    # output is emittance * bmag 
    
    # geometric emittance in transverse plane
    out1 = y_out[:, Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:, Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    emit = np.sqrt(out1 * out2)
  
    sigma_x = y_out[:, Model.loc_out['sigma_x']] #grab sigma_x out of the model 
    sigma_y = y_out[:, Model.loc_out['sigma_y']] #grab sigma_y out of the model 
    
    # real beta and alpha 
    # NEEDS TO BE FIXED - currently assuming real alpha to be the same as design alpha 
    alpha_x, alpha_y = alpha0_x, alpha0_y
    beta_x, beta_y = (sigma_x**2) / out1, (sigma_y**2) / out2
    
    # bmag 
    bmag_x = 0.5 * ((beta0_x / beta_x) + (beta_x / beta0_x)) + 0.5 * ((alpha_x * np.sqrt(beta0_x / beta_x) - alpha0_x * np.sqrt(beta_x / beta0_x))**2)
    bmag_y = 0.5 * ((beta0_y / beta_y) + (beta_y / beta0_y)) + 0.5 * ((alpha_y * np.sqrt(beta0_y / beta_y) - alpha0_y * np.sqrt(beta_y / beta0_y))**2)
    bmag = np.sqrt(bmag_x * bmag_y)
    #print(f'bmag: {bmag} emit: {emit}') 
    
    return (emit * bmag)/1e-6 # in um units 
    #return np.sqrt(out1*out2)/1e-6 # in um units

In [5]:
# mesh grid of 3^9 points to sample
n_samples_per_var = 3
n_var = 9

# create input and output data

var_points = torch.zeros((n_var, n_samples_per_var)) 
# take n_samples_per_var points from each dimension i
for i in range(n_var):
    var_points[i,:] = torch.as_tensor(np.linspace(bounds[i,0],bounds[i,1],n_samples_per_var))

# generate grid of points to sample 
grid = np.array(np.meshgrid(*var_points)).reshape(n_var,-1)
x = torch.zeros((n_samples_per_var**n_var, n_var))
for i in range(n_samples_per_var**n_var):
    x[i:] = torch.as_tensor(grid[:,i])

print(x)
print(x.shape)
y = torch.tensor([evaluate(vars) for vars in x])
print(y)

tensor([[ 0.4600, -0.0200, -0.0200,  ..., -1.0000, -1.0000, -7.0000],
        [ 0.4600, -0.0200, -0.0200,  ..., -1.0000, -1.0000, -3.0000],
        [ 0.4600, -0.0200, -0.0200,  ..., -1.0000, -1.0000,  1.0000],
        ...,
        [ 0.4850,  0.0200,  0.0200,  ...,  7.0000,  7.0000, -7.0000],
        [ 0.4850,  0.0200,  0.0200,  ...,  7.0000,  7.0000, -3.0000],
        [ 0.4850,  0.0200,  0.0200,  ...,  7.0000,  7.0000,  1.0000]])
torch.Size([19683, 9])


2022-06-16 22:37:16.739653: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-16 22:37:16.759383: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996175000 Hz


tensor([ -6.9780, -10.3164,  -5.1626,  ...,  -4.3536,  -6.9199,  -7.3886],
       dtype=torch.float64)


In [6]:
file = open('grid.py', 'w')
file.write(f'{y.tolist()}')
file.close()

## Fit data with second order polynomial

In [10]:
# f(x) = x^T A x + B x + C
# x is a 9x1 column vector, A is a symmetric 9x9 matrix, B is a 1x9 vector, C is a scalar.
a_vals = torch.randn((int(n_var*(n_var+1)/2),1)).reshape(1,-1)
A = torch.zeros(n_var, n_var)
i, j = torch.triu_indices(n_var, n_var)
A[i, j] = a_vals
A.T[i, j] = a_vals
A = A.clone().detach().requires_grad_(True)
#print(A)

B = torch.randn((1, n_var), requires_grad=True)
#print(B) 

C = torch.randn((1,1), requires_grad = True)
#print(C)

learning_rate = 1e-6
for i in range(2000):
    if (i > 30 and i % 10 == 0 and learning_rate >= 1e-7):
        learning_rate /= 10
    y_pred = torch.cat([torch.matmul(vars, torch.matmul(A, vars.T)) + torch.matmul(B, vars.T) + C for vars in x]).reshape(1,-1)[0]

    loss = torch.abs(y_pred - y).sum()
    if (i % 50 == 0):
        print(i, loss.item())
        print(y_pred)
    #print(loss.item())
    loss.backward()

    with torch.no_grad():
        A -= learning_rate * A.grad
        B -= learning_rate * B.grad
        C -= learning_rate * C.grad

        # Manually zero the gradients after updating weights
        A.grad = None
        B.grad = None
        C.grad = None

print(f'Result: y = x{A}X^T + {B} x + {C.item()}')

0 1180063.1125781015
tensor([ -83.8361,  -95.4777, -117.5265,  ...,  188.6398,  116.2071,
          33.3672], grad_fn=<SelectBackward0>)
50 268904.65776186064
tensor([-5.8543,  0.2384, -9.3206,  ...,  4.2198, 16.9607, 14.0498],
       grad_fn=<SelectBackward0>)
100 252950.68370055035
tensor([-6.7820, -0.7679, -9.9288,  ...,  3.4412, 15.7488, 12.8813],
       grad_fn=<SelectBackward0>)
150 239120.70548099093
tensor([ -6.6125,  -1.1285, -10.1580,  ...,   3.1692,  14.6991,  11.7155],
       grad_fn=<SelectBackward0>)
200 226727.53045599908
tensor([ -6.0840,  -1.2237, -10.1739,  ...,   2.6507,  13.4063,  10.3515],
       grad_fn=<SelectBackward0>)
250 215458.5689535886
tensor([ -5.4795,  -1.2521, -10.1094,  ...,   1.9944,  12.0123,   8.9455],
       grad_fn=<SelectBackward0>)
300 205284.92205806822
tensor([ -4.9037,  -1.2763, -10.0080,  ...,   1.1921,  10.5289,   7.5067],
       grad_fn=<SelectBackward0>)
350 196155.4677134361
tensor([-4.4805, -1.3828, -9.9258,  ...,  0.2865,  8.9957,  6.0

In [ ]:
test_x_1 = x[0] 
test_x_2 = x[1]
test_y_1 = torch.matmul(test_x_1, torch.matmul(A, test_x_1.T)) + torch.matmul(B, test_x_1.T) + C
test_y_2 = torch.matmul(test_x_2, torch.matmul(A, test_x_2.T)) + torch.matmul(B, test_x_2.T) + C
print(test_y_1)
print(test_y_2)

In [ ]:
print(hi)